In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
import pandas as pd
import plotly.express as px

from download import download


mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False


In [3]:
df = pd.read_csv('filtered_nn_data.csv')

In [4]:
df.head()

,date,year,uv,buyer,gmv,orders,cr,day_of_week,weekday,day_type,day_binary,campaign_info,campaign_type
0,2021-01-01,2021,96281,3595,66052,4331,0.037,4,1,Weekday,100,non campaign,non-campaign
1,2021-01-02,2021,93620,3852,68483,4599,0.041,5,0,Weekend,101,non campaign,non-campaign
2,2021-01-03,2021,98196,4402,75556,5254,0.045,6,0,Weekend,110,non campaign,non-campaign
3,2021-01-04,2021,108903,4374,75432,5292,0.040,0,1,Weekday,0,non campaign,non-campaign
4,2021-01-05,2021,123715,4486,74678,5518,0.036,1,1,Weekday,1,non campaign,non-campaign


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           748 non-null    object 
 1   year           748 non-null    int64  
 2   uv             748 non-null    int64  
 3   buyer          748 non-null    int64  
 4   gmv            748 non-null    int64  
 5   orders         748 non-null    int64  
 6   cr             748 non-null    float64
 7   day_of_week    748 non-null    int64  
 8   weekday        748 non-null    int64  
 9   day_type       748 non-null    object 
 10  day_binary     748 non-null    int64  
 11  campaign_info  748 non-null    object 
 12  campaign_type  748 non-null    object 
dtypes: float64(1), int64(8), object(4)
memory usage: 76.1+ KB


In [6]:
df.describe()

,year,uv,buyer,gmv,orders,cr,day_of_week,weekday,day_binary
count,748.000000,748.000000,748.000000,748.000000,748.000000,748.000000,748.000000,748.000000,748.000000
mean,2021.921123,159671.104278,6223.236631,101562.319519,7512.299465,0.039114,3.010695,0.709893,47.983957
std,0.769522,29478.107832,1239.675450,21527.881074,1471.932727,0.004362,2.003315,0.454115,48.897919
min,2021.000000,79290.000000,3105.000000,42574.000000,3718.000000,0.022000,0.000000,0.000000,0.000000
25%,2021.000000,139986.000000,5496.500000,88331.000000,6629.000000,0.037000,1.000000,0.000000,1.000000
50%,2022.000000,158205.500000,6147.500000,99430.000000,7442.500000,0.039000,3.000000,1.000000,11.000000
75%,2023.000000,181223.000000,7000.750000,111620.500000,8470.250000,0.041000,5.000000,1.000000,101.000000
max,2023.000000,286385.000000,12621.000000,291512.000000,15856.000000,0.058000,6.000000,1.000000,110.000000


In [7]:
df.columns

Index(['date', 'year', 'uv', 'buyer', 'gmv', 'orders', 'cr', 'day_of_week',
       'weekday', 'day_type', 'day_binary', 'campaign_info', 'campaign_type'],
      dtype='object')

In [8]:
df.shape

(748, 13)

In [9]:
df.shape[0]

748

In [10]:
df.shape[1]

13

In [13]:
df = df[['date', 'uv', 'buyer', 'gmv', 'orders']].copy()

KeyError: ('date', 'uv', 'buyer', 'gmv', 'orders')